In [172]:
from sklearn import preprocessing

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

import pycountry_convert

import plotly
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

init_notebook_mode(connected=True)

In [108]:
def iso2_to_countryname(iso2_code):
    iso2_code = iso2_code.lower()
    if iso2_code == 'uk':
        iso2_code = 'gb'
    if iso2_code == 'el':
        iso2_code = 'gr'
    if iso2_code == 'cs':
        iso2_code = 'rs'
    if iso2_code == 'xk':
        return 'Kosovo'
    if iso2_code == 'ko':
        return 'Kosovo'
    if iso2_code == 'an':
        return 'NETHERLANDS ANTILLES'
    return pycountry_convert.country_alpha2_to_country_name(iso2_code.upper())

# Data preparation

## H2020

In [83]:
h2020_projects = pd.read_csv('../data/processed/h2020-projects.csv', delimiter='\t', header=None)
h2020_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
h2020_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,195398,657139,stromamech,targeting stromal cells to modify tumor mechan...,151648.80,151648.80,current chemotherapeutic agents are potent eno...,current chemotherapeutic agents are potent eno...,2015-05-01,2017-04-30,signed
1,199945,691161,geo-safe,geospatial based environment for optimisation...,1386000.00,1080000.00,"in eu and australia, every year thousands of s...","in eu and australia, every year thousands of s...",2016-05-01,2020-04-30,signed
2,207081,738654,vr-energy,"a reliable, efficient, flexible and cost ffect...",2390000.00,1673000.00,hydraredox iberia s.l. is a spanish sme formed...,hydraredox iberia s.l. is a spanish sme formed...,2016-12-01,2018-11-30,signed
3,211829,756785,fun-pm,fundamental understanding of nanoparticle chem...,1493838.77,1493838.77,while modern societies are facing urgent chall...,while modern societies are facing urgent chall...,2018-02-01,2023-01-31,signed
4,212770,758865,bioele,functional biointerface elements via biomicrof...,1486938.00,1486938.00,"imagine in the future, bionic devices that can...","imagine in the future, bionic devices that can...",2018-01-01,2022-12-31,signed


In [84]:
h2020_organisations = pd.read_csv('../data/processed/h2020-organisations.csv', delimiter='\t', header=None)
h2020_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
h2020_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,195398,999835843,1905890.0,1,coordinator,ucy,university of cyprus,nicosia,cy,151648.8
1,199945,998929669,1908315.0,16,participant,upd,universite paris dauphine,paris cedex 16,fr,45000.0
2,199945,989393987,1931502.0,5,participant,efm,entente pour la forêt méditerranéenne,gardanne,fr,36000.0
3,199945,916450860,2791186.0,14,participant,uca,universite clermont auvergne,clermont-ferrand,fr,45000.0
4,199945,999974650,1906418.0,9,participant,unige,universite de geneve,geneve,ch,40500.0


## FP7

In [85]:
fp7_projects = pd.read_csv('../data/processed/fp7-projects.csv', delimiter='\t', header=None)
fp7_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp7_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,90524,239420,carnivorous zoo,carnivorous zooplankton – their role in swedis...,45000.00,45000.00,this project will focus on how carnivorous zoo...,this project will focus on how carnivorous zoo...,2009-04-01,2012-03-31,ong
1,91013,235569,toplacir,a two-photon survey of the plasticity of the n...,234427.01,234427.01,the cortex has been described to display synap...,the cortex has been described to display synap...,2009-06-01,2012-05-31,ong
2,107943,331428,spectra,"spectra of random matrices, graphs and groups",190113.60,190113.60,the goal is to understand the connections betw...,the goal is to understand the connections betw...,2013-07-15,2015-07-14,ong
3,89808,201751,arise,agricultural revolution in southern europe?,45000.00,45000.00,arise will provide a detailed survey of the di...,arise will provide a detailed survey of the di...,2007-10-01,2010-09-30,ong
4,90275,229761,episode,exploiting the potential of structural biology...,1084384.52,896991.00,episode’s goal is to maximize the regional ben...,episode’s goal is to maximize the regional ben...,2009-01-01,2011-12-31,ong


In [86]:
fp7_organisations = pd.read_csv('../data/processed/fp7-organisations.csv', delimiter='\t', header=None)
fp7_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp7_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,90524,999981925.0,1905548.0,1,coordinator,ugot,goeteborgs universitet,goeteborg,se,45000.00
1,91013,999997833.0,1906210.0,1,coordinator,inserm,institut national de la sante et de la recherc...,paris,fr,234427.01
2,107943,998078979.0,1909560.0,1,coordinator,renyi,magyar tudomanyos akademia renyi alfred matema...,budapest,hu,190113.60
3,89808,999978336.0,1905740.0,1,coordinator,york,university of york,york north yorkshire,uk,45000.00
4,90275,999753781.0,1922946.0,6,participant,fav,tsb innovationsagentur berlin gmbh,berlin,de,143980.00


## FP6

In [87]:
fp6_projects = pd.read_csv('../data/processed/fp6-projects.csv', delimiter='\t', header=None)
fp6_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp6_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,79861,23120,irriqual,sustainable orchard irrigation for improving f...,3684294,2249693,"european countries, especially in the mediterr...","european countries, especially in the mediterr...",2006-07-01,2009-12-31,NaN
1,85584,30379,lowjuice,novel process for reducing sugar and adding fi...,1948483,1185353,there are some 153.000 apple farmers and app. ...,there are some 153.000 apple farmers and app. ...,2006-09-01,2009-08-31,NaN
2,83990,43147,food-frenz,food research in europe and new zealand (food-...,500052,349999,sumer trends in food sumption are rapidly chan...,sumer trends in food sumption are rapidly chan...,2006-09-01,2008-08-31,NaN
3,80198,035164,scier,sensor and computing infrastructure for enviro...,3266949,2091000,"the scier project will design, develop, and de...","the scier project will design, develop, and de...",2006-07-01,2008-12-31,NaN
4,85062,39965,astrocenter,development of an astrophysics centre in crete,0,723100,"astrophysicists in crete, among the most activ...","astrophysicists in crete, among the most activ...",2006-09-01,2010-08-31,NaN


In [88]:
fp6_organisations = pd.read_csv('../data/processed/fp6-organisations.csv', delimiter='\t', header=None)
fp6_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp6_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,79861,NaN,975516,11,participant,lra,litani river authority,beirut,lb,NaN
1,79861,NaN,1361235,3,participant,fecoam,federacion de cooperativas agrarias de murcia,murcia,es,NaN
2,79861,NaN,1361237,4,participant,abacotech,abacotech s.l.,cartagena,es,NaN
3,79861,NaN,1361239,5,participant,contariego,contadores de riegos s.l.,"san gines, murcia",es,NaN
4,79861,NaN,1361241,6,participant,inra,institut national de la recherche agronomique,paris,fr,NaN


## FP5

In [89]:
fp5_projects = pd.read_csv('../data/processed/fp5-projects.csv', delimiter='\t', header=None)
fp5_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp5_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,58742,ica4-ct-2001-10061,cotonbiomat,processing cotton seed into biodegradable mate...,1235741.0,673408.0,"like most european countries, brazil and argen...","like most european countries, brazil and argen...",2001-11-01,2005-10-31,NaN
1,62872,ist-2001-65001,ims-noe,ims-noe,1522264.0,1398364.0,NaN,NaN,2002-06-01,2005-05-31,NaN
2,59524,hpcf-ct-2000-00196-01,NaN,13th international conference on ion implantat...,NaN,NaN,ion implantation technology (iit) conferences ...,ion implantation technology (iit) conferences ...,2000-09-15,2000-09-21,NaN
3,91742,qlk5-ct2002-51713,folgglo,functionality of l-gulono-gamma-lactone oxidas...,125900.0,125900.0,"vitamin c (ascorbic acid, aa) deficiency is on...","vitamin c (ascorbic acid, aa) deficiency is on...",2003-02-21,2005-02-20,NaN
4,64337,hpmt-ct-2001-00372,NaN,innovative ceramic processing,120000.0,120000.0,"the institut ""jozef stefan"", ceramics departme...","the institut ""jozef stefan"", ceramics departme...",2002-02-01,2006-01-31,NaN


In [90]:
fp5_organisations = pd.read_csv('../data/processed/fp5-organisations.csv', delimiter='\t', header=None)
fp5_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp5_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,58742,NaN,344587,1,coordinator,NaN,centre de coopération internationale en recher...,paris,fr,NaN
1,58742,NaN,858508,5,participant,NaN,incotec international b.v.,enkhuizen,nl,NaN
2,58742,NaN,858510,6,participant,NaN,national institute of industrial technology,san martin,ar,NaN
3,58742,NaN,858512,3,participant,NaN,ecole nationale supérieure des techniques indu...,ales,fr,NaN
4,58742,NaN,858514,2,participant,NaN,association pour la recherche et le developpem...,ales,fr,NaN


## FP4

In [91]:
fp4_projects = pd.read_csv('../data/processed/fp4-projects.csv', delimiter='\t', header=None)
fp4_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp4_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,35874,22574,kiss,kaizen information system and management support,3600000.0,1800000.0,"following the japanese lead, manufacturing org...","following the japanese lead, manufacturing org...",1996-10-01,1998-09-30,NaN
1,40808,fmbi950398,NaN,non linearity and quantum lattice fluctuations...,NaN,NaN,NaN,despite its crucial role in standard superco...,1996-03-01,1998-02-28,NaN
2,43751,jos3970021,NaN,voluntary agreements - implementation and effi...,NaN,NaN,NaN,during the last years voluntary agreements (...,1998-02-01,2000-01-31,NaN
3,42266,fmbi961657,NaN,development of automatic program comprehension...,NaN,NaN,NaN,research objectives and content this resea...,1997-06-01,1999-11-30,NaN
4,42537,fmbi972047,NaN,"functional aspects of ypka, a yersinia secreto...",NaN,NaN,research objectives and content the yersin...,research objectives and content the yersin...,1997-08-01,1999-07-31,NaN


In [92]:
fp4_organisations = pd.read_csv('../data/processed/fp4-organisations.csv', delimiter='\t', header=None)
fp4_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp4_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,35874,NaN,127879,6,participant,txt,txt e-solutions spa,milano,it,NaN
1,35874,NaN,503576,3,participant,NaN,efeso consulenza,milano,it,NaN
2,35874,NaN,551765,5,participant,NaN,"sema group, s.a.e.",madrid,es,NaN
3,35874,NaN,690124,1,coordinator,NaN,pirelli pneumatici spa,milano,it,NaN
4,35874,NaN,692717,4,participant,NaN,"intracom sa, hellenic telecommunications and e...",peania - attika,el,NaN


## FP3

In [93]:
fp3_projects = pd.read_csv('../data/processed/fp3-projects.csv', delimiter='\t', header=None)
fp3_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp3_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,5846,ev5v0270,NaN,desertification processes in the mediterranean...,NaN,NaN,to combine aerological measurements by means o...,to combine aerological measurements by means o...,1993-08-01,1996-01-31,NaN
1,26677,ts3*910021,NaN,nutrient cycling and sustainability in alley-c...,NaN,NaN,this project continues the work begun in our...,this project continues the work begun in our...,1992-04-01,1994-08-31,NaN
2,27340,mat1930012,NaN,preparation of candidate reference material ma...,NaN,NaN,groundwater is currently monitored by eu labor...,groundwater is currently monitored by eu labor...,1994-01-01,1997-07-31,NaN
3,4897,fi3p0052,NaN,patient dose from radiopharmaceuticals,NaN,NaN,"in recent years, legal requirements of nation...","in recent years, legal requirements of nation...",1992-09-01,1995-06-30,NaN
4,27411,mat1940054,NaN,intercomparison of laboratory measurements of ...,NaN,NaN,cen/tc126/wg1 is dealing with the development ...,cen/tc126/wg1 is dealing with the development ...,1994-12-01,1997-05-31,NaN


In [94]:
fp3_organisations = pd.read_csv('../data/processed/fp3-organisations.csv', delimiter='\t', header=None)
fp3_organisations.columns = ['rcn', 'org_id', 'org_rcn',  'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp3_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,5846,NaN,344843,2,participant,NaN,centre national de la recherche scientifique,toulouse,fr,NaN
1,5846,NaN,346052,3,participant,NaN,german aerospace centre,wessling,de,NaN
2,5846,NaN,355320,4,participant,NaN,universitaet hannover,hannover,de,NaN
3,5846,NaN,445788,1,coordinator,NaN,universität karlsruhe (technische hochschule),karlsruhe,de,NaN
4,26677,NaN,144996,1,coordinator,NaN,university of cambridge,cambridge,uk,NaN


## FP2

In [95]:
fp2_projects = pd.read_csv('../data/processed/fp2-projects.csv', delimiter='\t', header=None)
fp2_projects.columns = ['rcn', 'reference', 'acronym', 'title', 'total_cost', 'ec_contribution',
                    'teaser', 'objective', 'start', 'end', 'status']
fp2_projects.head()

,rcn,reference,acronym,title,total_cost,ec_contribution,teaser,objective,start,end,status
0,2634,mast0030,NaN,combined sensor and information technology for...,NaN,NaN,the main objective of the programme is to enha...,the main objective of the programme is to enha...,1990-11-01,1993-10-31,NaN
1,288,jour0047,NaN,pv systems research at the 50 kwp pv installat...,NaN,NaN,the objective of the project is to : - inves...,the objective of the project is to : - inves...,1990-04-01,1992-03-31,NaN
2,28473,geno0005,NaN,genetic analyses in aids-research,NaN,NaN,NaN,NaN,1990-12-01,1991-11-30,NaN
3,3203,breu0317,NaN,manufacturing cell operator's expert system - ...,NaN,NaN,the objective of the project are : 1)to intr...,the objective of the project are : 1)to intr...,1990-06-01,1993-05-31,NaN
4,2734,sc1*0456,NaN,dynamique du manteau terrestre et grandes defo...,NaN,NaN,what is driving the plates at the surface of...,what is driving the plates at the surface of...,1990-09-01,1994-06-30,NaN


In [96]:
fp2_organisations = pd.read_csv('../data/processed/fp2-organisations.csv', delimiter='\t', header=None)
fp2_organisations.columns = ['rcn', 'org_id', 'org_rcn', 'order', 'org_type',
                         'short_name', 'legal_name',
                         'city', 'country', 'contribution']
fp2_organisations.head()

,rcn,org_id,org_rcn,order,org_type,short_name,legal_name,city,country,contribution
0,2634,NaN,344286,2,participant,NaN,centro de estudios y experimentación de obras ...,madrid,es,NaN
1,2634,NaN,345310,3,participant,NaN,comex technologie sa,marseille,fr,NaN
2,2634,NaN,349555,5,participant,jülich,forschungszentrum juelich gmbh,juelich,de,NaN
3,2634,NaN,356382,1,coordinator,NaN,the university of liverpool,liverpool,uk,NaN
4,2634,NaN,358244,4,participant,NaN,demokritos university of thrace,komotini,el,NaN


# Data analysis

In [97]:
h2020_funds = h2020_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp7_funds = fp7_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp6_funds = fp6_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp5_funds = fp5_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp4_funds = fp4_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp3_funds = fp3_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

fp2_funds = fp2_organisations.groupby(['country'])\
                    .agg({'contribution': 'sum', 'rcn': 'count'})\
                    .sort_values(['contribution', 'rcn'], ascending=False)

In [208]:
data = h2020_funds.join(fp7_funds, rsuffix='_fp7', how='outer')
data = data.join(fp6_funds, rsuffix='_fp6', how='outer')
data = data.join(fp5_funds, rsuffix='_fp5', how='outer')
data = data.join(fp4_funds, rsuffix='_fp4', how='outer')
data = data.join(fp3_funds, rsuffix='_fp3', how='outer')
data = data.join(fp2_funds, rsuffix='_fp2', how='outer')
data

,contribution,rcn,contribution_fp7,rcn_fp7,contribution_fp6,rcn_fp6,contribution_fp5,rcn_fp5,contribution_fp4,rcn_fp4,contribution_fp3,rcn_fp3,contribution_fp2,rcn_fp2
country,,,,,,,,,,,,,,
ae,0.000000e+00,2.0,3.702199e+05,7.0,0.000000e+00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
af,7.530775e+05,2.0,1.671875e+04,1.0,0.000000e+00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ai,8.797625e+05,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
al,2.248823e+06,22.0,2.303318e+06,41.0,8.910000e+04,41.0,0.0,8.0,0.0,8.0,NaN,NaN,NaN,NaN
am,9.896775e+05,26.0,3.111579e+06,44.0,2.380000e+04,15.0,0.0,13.0,0.0,6.0,NaN,NaN,NaN,NaN
an,NaN,NaN,NaN,NaN,0.000000e+00,2.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ao,0.000000e+00,1.0,9.797500e+04,2.0,0.000000e+00,2.0,NaN,NaN,0.0,2.0,0.0,2.0,NaN,NaN
ar,5.010929e+06,122.0,1.455203e+07,122.0,5.316000e+04,97.0,0.0,50.0,0.0,44.0,0.0,13.0,0.0,2.0
at,1.085617e+09,2700.0,1.153225e+09,3651.0,3.420585e+07,1975.0,0.0,1771.0,0.0,1362.0,0.0,221.0,0.0,59.0


In [209]:
data = data.fillna(0)

In [210]:
data[data.index == 'pl']

,contribution,rcn,contribution_fp7,rcn_fp7,contribution_fp6,rcn_fp6,contribution_fp5,rcn_fp5,contribution_fp4,rcn_fp4,contribution_fp3,rcn_fp3,contribution_fp2,rcn_fp2
country,,,,,,,,,,,,,,
pl,3.586897e+08,1508.0,4.306081e+08,2253.0,9906841.14,1903.0,0.0,1158.0,0.0,275.0,0.0,8.0,0.0,0.0


In [211]:
data['total_contribution'] = data['contribution']\
                            + data['contribution_fp7']\
                            + data['contribution_fp6']\
                            + data['contribution_fp5']\
                            + data['contribution_fp4']\
                            + data['contribution_fp3']\
                            + data['contribution_fp2']
data['total_projects'] = data['rcn']\
                            + data['rcn_fp7']\
                            + data['rcn_fp6']\
                            + data['rcn_fp5']\
                            + data['rcn_fp4']\
                            + data['rcn_fp3']\
                            + data['rcn_fp2']
data = data[['total_contribution', 'total_projects']]

In [212]:
min_max_scaler = preprocessing.MinMaxScaler((3, 100))

In [213]:
data = data.sort_values(['total_contribution', 'total_projects'], ascending=False).reset_index()
trace = go.Scatter(x=data.index,
                   y=data['total_contribution'],
                   mode='markers',
                   text=data['country'],
                   marker=dict(size=min_max_scaler.fit_transform(data[['total_projects']]))
                  )

layout = go.Layout(
    title='',
    xaxis=dict(type='log'),
    yaxis=dict(type='log')
)

fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig)

In [214]:
data = data.sort_values(['total_projects', 'total_contribution'], ascending=False).reset_index()
trace = go.Scatter(x=data.index,
                   y=data['total_projects'],
                   mode='markers',
                   text=data['country'],
                   marker=dict(size=min_max_scaler.fit_transform(data[['total_contribution']]))
                  )

layout = go.Layout(
    title='',
    xaxis=dict(type='log'),
    yaxis=dict(type='log')
)

fig = dict(data=[trace], layout=layout)
plotly.offline.iplot(fig)

In [159]:
funding_map = [ dict(
        type = 'choropleth',
        locationmode = 'country names',
        locations = data['country'].apply(iso2_to_countryname),
        z = data['total_contribution'],
        text = data['total_contribution'],
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = 'Euro (Billions)'),
      ) ]

layout = dict(
    title = 'EU funding distribution',
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'equirectangular'
        )
    )
)

fig = dict(data=funding_map, layout=layout)
plotly.offline.iplot(fig, validate=False)

KeyError: "Invalid Country Alpha-2 code: 'YU'"

In [110]:
projects_map = [ dict(
        type = 'choropleth',
        locationmode = 'country names',
        locations = data['country'].apply(iso2_to_countryname),
        z = data['total_projects'],
        text = data['total_projects'],
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            title = '# projects'),
      ) ]

layout = dict(
    title = 'EU projects distribution',
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'equirectangular'
        )
    )
)

fig = dict(data=projects_map, layout=layout)
plotly.offline.iplot(fig, validate=False)

KeyError: "Invalid Country Alpha-2 code: 'EU'"